In [ ]:
# Workaround for training output not visible in JupyterNotebook https://github.com/microsoft/vscode-jupyter/issues/13163
from IPython.display import clear_output, DisplayHandle
def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [ ]:
from fastai.vision.all import *
import pandas as pd

In [ ]:
foodPath = untar_data(URLs.FOOD)

In [ ]:
get_image_files(foodPath)[0:15]

In [ ]:
print(os.listdir(foodPath))

In [ ]:
image_dir_path = foodPath/'images'
image_categories = os.listdir(image_dir_path)
print(image_categories[0:10])

In [ ]:
img = PILImage.create('/home/mendhak/.fastai/data/food-101/images/tiramisu/63187.jpg')
img.show();

In [ ]:
df_train=pd.read_json('/home/mendhak/.fastai/data/food-101/train.json')
df_train.head()

In [ ]:
df_test=pd.read_json('/home/mendhak/.fastai/data/food-101/test.json')
df_test.head()

In [ ]:
labelA = 'cheesecake'
labelB = 'donuts'
labelC = 'panna_cotta'

In [ ]:
for img in get_image_files(foodPath): 
  if labelA in str(img):
   img.rename(f"{img.parent}/{labelA}-{img.name}")
  elif labelB in str(img):
    img.rename(f"{img.parent}/{labelB}-{img.name}")
  elif labelC in str(img):
    img.rename(f"{img.parent}/{labelC}-{img.name}")
  else: os.remove(img)

In [ ]:
len(get_image_files(foodPath))

In [ ]:
def GetLabel(fileName):
 return fileName.split('-')[0]

GetLabel("cheesecake-1092082.jpg")

In [ ]:
dls = ImageDataLoaders.from_name_func(
    foodPath, 
    get_image_files(foodPath), 
    valid_pct=0.2, 
    seed=42,
    label_func=GetLabel, 
    item_tfms=Resize(224))

dls.train.show_batch()

In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy, pretrained=True)
learn.fine_tune(epochs=10)

In [ ]:
url_to_image = input()
print(url_to_image)
from fastdownload import download_url
dest = 'user_input.jpg'
download_url(url_to_image, dest, show_progress=False)

img = PILImage.create(dest)
img.show()

label,_,probs = learn.predict(img)

print(f"This is a {label}.")
print(f"{labelA} {probs[0].item():.2f}")
print(f"{labelB} {probs[1].item():.2f}")
print(f"{labelC} {probs[2].item():.2f}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for i in range(0,10):
  randomIndex = random.randint(0, len(get_image_files(foodPath))-1)
  img = mpimg.imread(get_image_files(foodPath)[randomIndex])
  label,_,probs = learn.predict(img)

  fig = plt.figure()
  ax = fig.add_subplot() 
  imgplot = plt.imshow(img) 
  ax.set_title(label) 

  plt.gca().axes.get_yaxis().set_visible(False)
  plt.gca().axes.get_xaxis().set_visible(False)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.plot_top_losses(4,nrows=4)

In [ ]:
learn.export() 
modelPath = get_files(foodPath, '.pkl')[0]
modelPath

In [ ]:
learn_inf = load_learner(modelPath)
learn_inf.predict(mpimg.imread(get_image_files(foodPath)[1260])) 